# Capstone Notebook

This Notebook is created for the Applied Data Science Capstone project

In [1]:
import pandas as pd
import numpy as np


In [4]:
print('Hello Capstone Project Course')

Hello Capstone Project Course


Importing packages needed

In [2]:
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json


from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [3]:
data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

## Scaper

In [4]:
data_html = pd.read_html(data_url)

In [5]:
Neighborhoods = data_html[0]
Neighborhoods.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [6]:
Neighborhoods.dropna(inplace = True, axis = 0)

In [7]:
Neighborhoods.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
Neighborhoods.shape

(103, 3)

# Adding Latitudes and Longitudes

In [9]:
df_ll = pd.read_csv('http://cocl.us/Geospatial_data')

In [10]:
la = []
lo = []
for a in Neighborhoods['Postalcode']:
    temp_index = df_ll[df_ll['Postal Code'].eq(a)]
    temp_la = temp_index['Latitude'].values
    temp_lo = temp_index['Longitude'].values
    la.append(temp_la[0])
    lo.append(temp_lo[0])
Neighborhoods['Latitude'] = la
Neighborhoods['Longitude'] = lo

In [11]:
Neighborhoods.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


# Creating a map and adding markers for neighborhoods in Toronto

In [12]:
address = ' Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Neighborhoods['Latitude'], Neighborhoods['Longitude'], Neighborhoods['Borough'], Neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Slicing the data frame to get neighborhoods in Downtown Toronto

In [14]:
downtown_df = Neighborhoods[Neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop = True)
downtown_df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,Garden District / Ryerson,43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [15]:
downtown_address = 'Downtown Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(downtown_address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [16]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)
for lat, lng, label in zip(downtown_df['Latitude'], downtown_df['Longitude'], downtown_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

## Using Foursquare api to explore

In [17]:
CLIENT_ID = 'SZPQQRN10VAR4YQTKNJLJLWFE5KU40KPSELLPFVV3IQCYTJM'
CLIENT_SECRET = 'QSGXPFTU2KHLWWKVNQ3MS11NWBMJRCMWMMKTQRN0TT5I4CVL'
VERSION = '20180605'

##Getting latitude and longitude of Rosedale to explore using the API

In [18]:
neighborhood_latitude = downtown_df.loc[0, 'Latitude']
neighborhood_longitude = downtown_df.loc[0, 'Longitude'] 
neighborhood_name = downtown_df.loc[0, 'Neighborhood']

Gettting top 7 places in Rosedale

In [19]:
radius = 1200
LIMIT = 8
api_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(api_url).json()

## Extracting the list of venues

In [36]:
venues_temp = results['response']['groups']
venue_dic = venues_temp[0]
venue_t2 = venue_dic['items']
venue_t2 = venue_t2[1:]
venues = []
for v in venue_t2:
    venues.append(v['venue']['name'])

### Top 7 Places to visit in Rosedale

In [39]:
n = 1
for i in venues:
    print('{}. {}'.format(n,i))
    n+=1

1. Tandem Coffee
2. Impact Kitchen
3. The Distillery Historic District
4. Distillery Sunday Market
5. Corktown Common
6. Souk Tabule
7. SOMA chocolatemaker
